# 模型——卷积神经网络

卷积神经网络(Convolutional Neural Networks, CNN)与全连接神经网络一样，都可以看成：

- 可微分的函数，从输入空间映射到输出空间（经过一系列点乘和非线性操作）；
- 由神经元与需要学习的权重和偏置组成。

　　下图是全连接神经网络和卷积神经网络的图，可以很直观的看到神经元的组织结构。全连接神经网络的神经元是向量的形式组织的；而2D卷积神经网络的神经元是以三维(width, height, and depth)张量的形式组织的，也可以说是特征图。

![CNN_Neurons](./CNN_Neurons.png)

　　一个简单的卷积网络一般由一系列的层链接而成，每个层将一个三维张量转换成另一个三维张量，并且每个层的操作都必须是可微分的。一般卷积神经神经网络都包括：卷积层(Convolutional Layer)、池化层(Pooling Layer)和全连接层(Fully-connected Layer)。

    一个简单的卷积网络结构：[INPUT - CONV - RELU - POOL -FC]。



## １、卷积层

**卷积层的参数**由一系列的可以学习的滤波器(filters)或者卷积核构成。每个滤波器的权重是一个三维张量(width, height, and depth)，深度由输入张量的深度(通道数)决定。

**卷积层有四个超参数**：

         1. 输出张量的深度或者通道数(the depth of the output volume) D；
         2. 滤波器(Filter)或者卷积核的尺寸F；
         3. 卷积步长(Stride) S，通常为2；
         4. 边缘补0数量(Pad)，可以控制输出张量的尺寸。

- 卷积操作

  例子:滤波器尺寸为3，输出通道数为2，卷积步长为2，pad为１。

  进行卷积操作时，每个滤波器沿着输入张量的宽和高的方向移动，做点乘计算，产生一张二维的特征图。多个滤波器卷积产生的二维特征图叠起来构成三维的输出张量。

  ![Conv_op_dynamic](./Conv_op_dynamic.gif)

  

  ![Conv_op_1](./Conv_op_1.png)
  $$
  O[0, 0, 0]=X[:3,:3,0]*w0[:,:,0]+X[:3,:3,1]*w0[:,:,1]+X[:3,:3,2]*w0[:,:,2]+b0
  $$

 ![Conv_op_2](./Conv_op_2.png)
 $$
  O[1, 0, 0]=X[2:5,:3,0]*w0[:,:,0]+X[2:5,:3,1]*w0[:,:,1]+X[2:5,:3,2]*w0[:,:,2]+b0
  $$
 ![Conv_op_3](./Conv_op_3.png)
 $$
  O[2, 0, 0]=X[4:,:3,0]*w0[:,:,0]+X[4:,:3,1]*w0[:,:,1]+X[4:,:3,2]*w0[:,:,2]+b0
  $$
   ... ...

- 神经网络角度看卷积操作

  ![Conv_op](./Conv_op.png)

  ​      卷积层与全连接层主要的不同在于：卷积层的神经元间只是局部链接，输出的神经元只与输入的部分神经元有关。

- 输出张量(特征图)的尺寸计算

  ​      输入张量的尺寸为W1×H1×D1，滤波器尺寸为F，滤波器个数为K，卷积步长为S，补0数量为P，输出张量的尺寸为：

  - W2 = (W1-F+2P)/S+1
  - H2 = (H1-F+2P)/S+1
  - D2 = K


## 2、池化层

  卷积网络中通常会在两个卷积层之间加上一个池化层，用于减小隐藏层神经元的数量，从而减小计算量，同时还能控制过拟合。

 池化层没有可学习的参数，主要有**两个超参数**：

1. 池化窗口的尺寸F；
2. 步长(Stride) S，通常为2；

池化操作

  池化操作单独的作用于每张特征图(通道)，池化窗口沿着特征图的宽和高的方向移动，做Max操作。通常情况下，设置池化层的窗口尺寸为2，步长为2。

  ![Pooling](./Pooling.png)

- 输出特征图的尺寸计算

  输入张量的尺寸为W1×H1×D1，窗口尺寸为F，窗口步长为S，输出张量的尺寸为：

  - W2 = (W1-F)/S+1
  - H2 = (H1-F)/S+1
  - D2 = D1

## 3、pytorch搭建CNN

尝试为OxFlower17数据集搭建CNN分类器，输入图片格式为128×128×3，输出分类为17。
pytorch主要有两种方式搭建模型：

## 3.1 torch.nn.Sequential 快速搭建
torch.nn.Sequential是一个有序的容器，将神经网络模块按照顺序传入构造器，这些模块将依次被添加到计算图中执行。

In [1]:
import torch.nn as nn
import torch.nn.functional as F


net1 = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1), 
    nn.ReLU(),
    nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3), 
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3), 
    nn.ReLU(),
    nn.MaxPool2d(2, 2), # 25*25*12
    nn.Linear(30*30*64, 512),
    nn.ReLU(),
    nn.Linear(512, 17),
    nn.Softmax()
)
print(net1)

Sequential(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU()
  (7): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (8): ReLU()
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Linear(in_features=57600, out_features=512, bias=True)
  (11): ReLU()
  (12): Linear(in_features=512, out_features=17, bias=True)
  (13): Softmax()
)


## 3.2　继承torch.nn.Module
通过添加属性的方式搭建神经网络各层的结构信息，在forward方法中完善神经网络各层之间的连接信息。

In [3]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1) # 128*128*32
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3) # 126*126*32
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1) # 63*63*64
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3) # 61*61*64
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(30*30*64, 512)
        self.fc2 = nn.Linear(512, 17)
        print(list(self.conv1.parameters()))
        

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.pool(F.relu(self.conv4(x)))
        x = x.view(-1, 30*30*64)
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=-1)
        return x

net2 = Net()
print(net2)

[Parameter containing:
tensor([[[[ 1.1337e-01, -1.2465e-01, -7.7205e-02],
          [-5.1657e-03, -4.5277e-03, -3.2635e-02],
          [-2.9369e-02, -1.5658e-01, -4.5013e-02]],

         [[-6.1500e-03, -1.7979e-01, -8.0260e-02],
          [ 5.5189e-02,  1.8506e-02,  5.5704e-02],
          [ 1.5733e-01,  1.2443e-01,  1.5873e-01]],

         [[-6.1944e-02,  1.8660e-01,  1.3538e-01],
          [-1.5532e-01, -1.0164e-01,  1.7240e-02],
          [ 9.2518e-02, -1.4552e-01, -1.1398e-01]]],


        [[[ 1.7156e-01, -1.7867e-01,  4.9703e-02],
          [ 1.6682e-01, -1.5402e-01, -6.9524e-02],
          [-9.0162e-02,  1.5123e-01, -8.4607e-02]],

         [[-1.2383e-01,  9.5216e-02, -2.8576e-02],
          [-1.0451e-02,  9.4307e-02, -2.3697e-02],
          [ 7.7163e-02,  1.0754e-01,  1.1126e-01]],

         [[-8.2547e-02, -4.5309e-02,  1.2254e-01],
          [ 1.7069e-01, -1.5850e-01,  1.4549e-01],
          [ 2.6857e-02, -6.0058e-02, -3.8783e-02]]],


        [[[-1.5258e-02,  1.2674e-01,  4.763

查看模型参数：

In [3]:
params = list(net1.parameters())
print(params)

[Parameter containing:
tensor([[[[ 0.0462,  0.0615, -0.0910,  0.0678,  0.0810, -0.0225],
          [ 0.0340, -0.0124, -0.0309,  0.0400, -0.0206,  0.0267],
          [ 0.0905, -0.0290,  0.0635,  0.0853, -0.0575,  0.0586],
          [ 0.0176, -0.0797,  0.0909,  0.0851, -0.0632,  0.0038],
          [ 0.0802,  0.0617,  0.0291, -0.0915,  0.0422,  0.0686],
          [-0.0211, -0.0497, -0.0416,  0.0497, -0.0426,  0.0488]],

         [[-0.0848,  0.0047,  0.0014, -0.0043, -0.0309, -0.0922],
          [ 0.0019, -0.0634, -0.0404, -0.0288,  0.0489, -0.0716],
          [ 0.0775, -0.0662, -0.0097,  0.0170, -0.0270, -0.0148],
          [-0.0941, -0.0875, -0.0519, -0.0506,  0.0260, -0.0925],
          [-0.0334, -0.0917,  0.0758,  0.0481, -0.0700,  0.0697],
          [ 0.0479, -0.0538,  0.0717,  0.0485, -0.0346, -0.0544]],

         [[ 0.0260,  0.0046,  0.0898, -0.0506,  0.0120,  0.0796],
          [-0.0191,  0.0169,  0.0286,  0.0299,  0.0145, -0.0061],
          [-0.0942, -0.0512,  0.0652,  0.0707, -0

## 补充softmax的介绍

Softmax层是一个非线性操作的层，它没有可以学习的参数。Softmax层作用是接收一个实数向量将其转换为概率分布的形式，其定义如下：

$$
softmax(x)=\frac{\exp(x_j)}{\sum_j exp(x_j)}, j=1, 2, ..., n
$$

返回结果为一个所有元素总和为1的向量，正好表示一个概率分布。